In [1]:
import os
import glob
import pandas as pd
import re
from tqdm.auto import tqdm

/home/wannaphong/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
list_all_txt_files=list(glob.glob(os.path.join(".","data","*","*","*","*.txt")))

In [3]:
len(list_all_txt_files)

30380

In [4]:
def read_file(path):
    text={"title":"","context":"","url":""}
    page_view_line = 0
    with open(path, 'r', encoding='utf-8-sig') as f:
        for n, line in enumerate(f):
            line = line.strip()
            if n == 0: # title line
                text["title"] = line.strip()
            else:
                if line:
                    if re.match(r'^[\d,]+$', line):
                        page_view_line = n
                        continue
                    if line == 'พิมพ์' or page_view_line and page_view_line < n: # skip 'print'
                        continue
                    if re.match(r'^ที่มา : http', line):
                        text["url"] = line.strip().split(" ")[-1]
                    else:
                        text["context"] +=line.strip().replace("\xa0","")+"\n"
    return text

In [5]:
print(read_file(list_all_txt_files[-1]))

{'title': 'รัฐบาลไทย-ข่าวทำเนียบรัฐบาล-นายกรัฐมนตรี ยกสถานการณ์โควิด-19 ที่ผ่านมาเป็นบทเรียน พร้อมวิเคราะห์ เรียนรู้ต่อเนื่อง เพื่อรับมือกับสถานการณ์ในปัจจุบัน', 'context': 'วันเสาร์ที่ 6 กุมภาพันธ์ 2564\nนายกรัฐมนตรี ยกสถานการณ์โควิด-19 ที่ผ่านมาเป็นบทเรียน พร้อมวิเคราะห์ เรียนรู้ต่อเนื่อง เพื่อรับมือกับสถานการณ์ในปัจจุบัน\nนายกรัฐมนตรี ยกสถานการณ์โควิด-19 ที่ผ่านมาเป็นบทเรียน พร้อมวิเคราะห์ เรียนรู้ต่อเนื่อง เพื่อรับมือกับสถานการณ์ในปัจจุบัน\nวันนี้ (6 กุมภาพันธ์ 2564) เวลา 8.30 น. พลเอก ประยุทธ์ จันทร์โอชา นายกรัฐมนตรีและรัฐมนตรีว่าการกระทรวงกลาโหม พูดคุยถึงการบริหารจัดการกับการระบาดใหม่ของโควิด-19 ในประเทศ ผ่าน PM PODCAST สรุปประเด็นดังนี้\nนายกรัฐมนตรีกล่าวถึงการระบาดโควิด-19 ในประเทศครั้งนี้ ไม่ได้เชื่อมโยงกับการระบาดในรอบแรก ที่ไทยสามารถควบคุมการแพร่ระบาดของโควิด-19 ได้ภายในสองเดือน การระบาดในรอบปัจจุบันส่งผลกระทบต่อระบบเศรษฐกิจอย่างต่อเนื่อง จึงต้องนำสถานการณ์ที่ผ่านมาเป็นบทเรียน วิเคราะห์ ประมวลสถานการณ์อย่างต่อเนื่อง ทำให้เกิดการเรียนรู้ เชื่อมั่น สามารถรับมือกับสถานการณ์ดังก

In [6]:
title=[]
context=[]
url=[]
for i in tqdm(list_all_txt_files):
    d=read_file(i)
    title.append(d["title"])
    context.append(d["context"])
    url.append(d["url"])

100%|███████████████████████████████████| 30380/30380 [00:05<00:00, 5841.57it/s]


In [7]:
df = pd.DataFrame.from_dict(
    {
        "title": title,
        "context": context,
        "url": url,
    }
)

In [8]:
df

,title,context,url
0,รัฐบาลไทย-ข่าวทำเนียบรัฐบาล-โปร 10 เดือน 10 กล...,วันศุกร์ที่ 9 ตุลาคม 2563\nโปร 10 เดือน 10 กลั...,https://www.thaigov.go.th/news/contents/detail...
1,รัฐบาลไทย-ข่าวทำเนียบรัฐบาล-“จับกัง 1” ติดตาม ...,วันศุกร์ที่ 9 ตุลาคม 2563\n“จับกัง 1” ติดตาม “...,https://www.thaigov.go.th/news/contents/detail...
2,รัฐบาลไทย-ข่าวทำเนียบรัฐบาล- ปลัดฯกอบชัย เปิดง...,วันศุกร์ที่ 9 ตุลาคม 2563\nปลัดฯกอบชัย เปิดงาน...,https://www.thaigov.go.th/news/contents/detail...
3,รัฐบาลไทย-ข่าวทำเนียบรัฐบาล-ปัญหาการเข้าถึงข้อ...,วันศุกร์ที่ 9 ตุลาคม 2563\nปัญหาการเข้าถึงข้อม...,https://www.thaigov.go.th/news/contents/detail...
4,รัฐบาลไทย-ข่าวทำเนียบรัฐบาล-SME D Bank ผนึกหน่...,วันศุกร์ที่ 9 ตุลาคม 2563\nSME D Bank ผนึกหน่ว...,https://www.thaigov.go.th/news/contents/detail...
...,...,...,...
30375,รัฐบาลไทย-,วันพฤหัสบดีที่ 1 มกราคม 2513\n,https://www.thaigov.go.th/news/contents/detail...
30376,รัฐบาลไทย-ข่าวทำเนียบรัฐบาล-นายกฯ เป็นห่วงประช...,วันเสาร์ที่ 6 กุมภาพันธ์ 2564\nนายกฯ เป็นห่วงป...,https://www.thaigov.go.th/news/contents/detail...
30377,รัฐบาลไทย-ข่าวทำเนียบรัฐบาล-รมว.ทส. เยี่ยมชมพิ...,วันเสาร์ที่ 6 กุมภาพันธ์ 2564\nรมว.ทส. เยี่ยมช...,https://www.thaigov.go.th/news/contents/detail...
30378,รัฐบาลไทย-ข่าวทำเนียบรัฐบาล-โควิดไม่หยุดโคดดิ้...,วันเสาร์ที่ 6 กุมภาพันธ์ 2564\nโควิดไม่หยุดโคด...,https://www.thaigov.go.th/news/contents/detail...


In [9]:
df.to_parquet("dataset.parquet", row_group_size=100, engine="pyarrow")

In [10]:
from datasets import Dataset

ds = Dataset.from_parquet("dataset.parquet")

Extracting data files: 100%|█████████████████████| 1/1 [00:00<00:00, 751.26it/s]
                                                                   

Dataset parquet downloaded and prepared to /home/wannaphong/.cache/huggingface/datasets/parquet/default-48a558d6a5154942/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


In [11]:
ds.push_to_hub("pythainlp/thaigov-v2-corpus-22032023")

Creating parquet from Arrow format: 100%|███████| 31/31 [00:01<00:00, 30.72ba/s]

Pushing dataset shards to the dataset hub: 100%|█| 1/1 [01:47<00:00, 107.37s/it]
